In [ ]:
import marimo as mo

In [ ]:
import os
import pathlib
from typing import Self
import pydantic
import networkx as nx
from ortools.sat.python import cp_model

# シフト最適化問題

## 問題設定

各ジョブは予め実行時間が与えられている.
その上で作業員を各ジョブに割り当て, 同じ作業員が同一時刻に複数のタスクを処理しないようにする.

### 定数

- $J$: ジョブの集合
- $W$: 作業員の集合
- $b_w$: 作業員 $w$ の固定費用
- $s_j, f_j$: ジョブ $j$ の開始時刻と終了時刻
- $J_w$: 作業員 $w$ が遂行できるジョブの集合
- $W_j$: ジョブ $j$ を遂行できる作業員の集合
- $G = (J, E)$: ジョブを点, ジョブの作業時間に重なりがあるとき点の間に線を張るというルールで作成したグラフ
- $C$: 区間グラフ $G$ に対する極大クリークの集合
- $J_c$: クリーク $c$ に含まれるジョブの集合

### 決定変数

- $x_{jw} \in \{ 0, 1 \}$: ジョブ $j$ に作業員 $w$ を割り当てる時 $1$, それ以外のとき $0$.
- $y_w \in \{ 0, 1 \}$: 作業員 $w$ が使われるとき $1$, それ以外のとき $0$.

### 制約条件

- $\sum_{w \in W} x_{jw} = 1 \quad (\forall j \in J)$: 各ジョブは 1 人の作業員によって必ず遂行される.
- $\sum_{j \in J_c \cap J_w} x_{jw} \leq y_w \quad (\forall w \in W, \space \forall c \in C)$: 各作業員は同時に 2 つ以上のジョブを処理できない.

### 目的関数

- $\min \sum_{w \in W} b_w y_w$: 作業員の固定費用の総和を最小化

In [ ]:
current_dir = pathlib.Path(os.path.dirname(__file__))
data_dir = current_dir / "data" / "ptask"

In [ ]:
fname1 = data_dir / "data_10_51_111_66.dat"

with open(fname1) as f:
    _lines = f.readlines()

_lines

In [ ]:
class Job(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)

    start: int
    finish: int

    def intersect_with(self, other: Self) -> bool:
        if self.start <= other.start and self.finish > other.start:
            return True
        if other.start <= self.start and other.finish > self.start:
            return True

        return False

In [ ]:
class Worker(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)

    cost: int
    jobs: list[int]

In [ ]:
class Condition(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)

    jobs: list[Job]
    workers: list[Worker]

    @staticmethod
    def from_file(path: str | os.PathLike) -> Self:
        jobs = []
        workers = []

        with open(path) as f:
            lines = f.readlines()
        idx = 0
        num_jobs = None
        while idx < len(lines):
            line = lines[idx]
            if line.startswith("Jobs = "):
                num_jobs: int = int(line.split()[2])
                print(f"{num_jobs=}")
                for _ in range(num_jobs):
                    idx += 1
                    line = lines[idx]
                    start, finish = line.strip().split()
                    jobs.append(Job(start=start, finish=finish))
            if line.startswith("Qualifications = "):
                num_workers: int = int(line.split()[2])
                print(f"{num_workers=}")
                for _ in range(num_workers):
                    idx += 1
                    line = lines[idx]
                    bw, jw = line.strip().split(":")
                    workers.append(Worker(cost=bw, jobs=jw.strip().split()))

            idx += 1

        return Condition(jobs=jobs, workers=workers)

In [ ]:
cond1 = Condition.from_file(fname1)

num_jobs=111
num_workers=51


In [ ]:
class Model:
    def __init__(self, cond: Condition):
        # グラフの定義
        g = nx.Graph()
        for id_job1, job1 in enumerate(cond.jobs):
            for id_job2, job2 in enumerate(cond.jobs):
                if id_job2 <= id_job1:
                    continue
                if job1.intersect_with(job2):
                    g.add_edge(id_job1, id_job2)

        # クリークの列挙
        cliques = [set(c) for c in nx.find_cliques(g)]

        model = cp_model.CpModel()

        x = [[model.new_bool_var("") for _ in cond.workers] for _ in cond.jobs]
        y = [model.new_bool_var("") for _ in cond.workers]

        # x と y の関係
        for idw, worker in enumerate(cond.workers):
            model.add_max_equality(
                y[idw],
                [x[idj][idw] for idj, _ in enumerate(cond.jobs)],
            )

        # worker ごとに担当できる job はきまっている
        for idj, job in enumerate(cond.jobs):
            for idw, worker in enumerate(cond.workers):
                if idj not in worker.jobs:
                    model.add(x[idj][idw] == 0)

        # 各 job は 1 人の worker によって処理される
        for idj, _ in enumerate(cond.jobs):
            model.add_exactly_one(
                [x[idj][idw] for idw, _ in enumerate(cond.workers)]
            )

        # 同一クリークに属する job は同じ worker で処理できない
        for clique in cliques:
            for idw, worker in enumerate(cond.workers):
                model.add_at_most_one([x[idj][idw] for idj in clique])

        # model.minimize(
        #     sum(
        #         y[idw]
        #         for idw, _ in enumerate(cond.workers)
        #     )
        # )

        # 解説ページでは worker 数を最小化していたが,
        # worker のコスト込で最小化するのが正しい.
        model.minimize(
            sum(
                y[idw] * worker.cost for idw, worker in enumerate(cond.workers)
            )
        )

        self.model = model
        self.x = x
        self.y = y
        self.cliques = cliques

    def solve(self, timeout: int = 10):
        self.solver = cp_model.CpSolver()
        self.solver.parameters.log_search_progress = True
        self.solver.parameters.max_time_in_seconds = timeout
        self.status = self.solver.solve(self.model)

In [ ]:
model1 = Model(cond1)
model1.solve()

mo.md(f"Optimal Value = {model1.solver.objective_value}")


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 10 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xf10b473897d169be)
#Variables: 5'712 (#bools: 51 in objective) (3'892 primary variables)
  - 5'712 Booleans in [0,1]
#kAtMostOne: 1'632 (#literals: 62'169)
#kExactlyOne: 111 (#literals: 5'661)
#kLinMax: 51 (#expressions: 5'661)
#kLinear1: 1'769

Starting presolve at 0.00s


  2.30e-03s  0.00e+00d  [DetectDominanceRelations] 
  1.58e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  8.69e-05s  0.00e+00d  [ExtractEncodingFromLinear] #potential_supersets=1'743 
  1.60e-03s  0.00e+00d  [DetectDuplicateColumns] 
  1.89e-04s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=47 
[Symmetry] Graph for symmetry has 15'294 nodes and 61'059 arcs.
[Symmetry] Symmetry computation done. time: 0.002689 dtime: 0.00593729
[SAT presolve] num removable Booleans: 0 / 3943
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:3892 literals:7784 vars:3943 one_side_vars:3943 simple_definition:0 singleton_clauses:0
[SAT presolve] [0.000114948s] clauses:3892 literals:7784 vars:3943 one_side_vars:3943 simple_definition:0 singleton_clauses:0
[SAT presolve] [0.000170423s] clauses:3892 literals:7784 vars:3943 one_side_vars:3943 simple_definition:0 singleton_clauses:0


  1.19e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.59e-02s  4.04e-02d  [Probe] #probed=7'886 


  2.68e-02s  1.48e-01d  [MaxClique] Merged 5'477(49'281 literals) into 1'183(31'779 literals) at_most_ones. 
  1.61e-03s  0.00e+00d  [DetectDominanceRelations] 
  3.59e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  7.48e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.12e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  7.83e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.99e-05s  0.00e+00d  [DetectDominatedLinearConstraints] 
  1.98e-05s  0.00e+00d  [DetectDifferentVariables] 
  1.52e-03s  1.21e-04d  [ProcessSetPPC] #relevant_constraints=1'294 
  3.14e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  2.41e-03s  3.17e-03d  [FindBigAtMostOneAndLinearOverlap] 
  2.65e-04s  1.79e-04d  [FindBigVerticalLinearOverlap] 
  7.94e-05s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  6.66e-05s  0.00e+00d  [MergeClauses] 


  1.63e-03s  0.00e+00d  [DetectDominanceRelations] 
  3.48e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.60e-03s  0.00e+00d  [DetectDominanceRelations] 
  3.42e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  9.09e-04s  0.00e+00d  [DetectDuplicateColumns] 
  9.16e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 7'057 nodes and 35'722 arcs.
[Symmetry] Symmetry computation done. time: 0.00119884 dtime: 0.0028611


...

#32      1.67s best:2991  next:[2989,2990] rnd_cst_lns (d=5.65e-01 s=112 t=0.10 p=0.55 stall=10 h=base)


#33      2.10s best:2990  next:[2989,2989] rnd_cst_lns (d=4.41e-01 s=138 t=0.10 p=0.50 stall=2 h=base)


#Model   2.58s var:3069/3943 constraints:1039/1294


#34      2.63s best:2989  next:[]         graph_var_lns (d=4.32e-01 s=165 t=0.10 p=0.47 stall=5 h=base)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   2.48s,    2.48s]    2.48s   0.00ns    2.48s         1 [   6.70s,    6.70s]    6.70s   0.00ns    6.70s
           'default_lp':         1 [   2.48s,    2.48s]    2.48s   0.00ns    2.48s         1 [   5.12s,    5.12s]    5.12s   0.00ns    5.12s
     'feasibility_pump':         1 [ 34.35ms,  34.35ms]  34.35ms   0.00ns  34.35ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         1 [  6.72ms,   6.72ms]   6.72ms   0.00ns   6.72ms         1 [  1.68ms,   1.68ms]   1.68ms   0.00ns   1.68ms
            'fs_random':         1 [  9.88ms,   9.88ms]   9.88ms   0.00ns   9.88ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
fname2 = data_dir / "data_40_138_360_33.dat"
cond2 = Condition.from_file(fname2)

num_jobs=360
num_workers=138


In [ ]:
model2 = Model(cond2)
model2.solve(timeout=60)

mo.md(f"Optimal Value = {model2.solver.objective_value}")


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12


Initial optimization model '': (model_fingerprint: 0x3a865ba7c720b275)
#Variables: 49'818 (#bools: 138 in objective) (16'804 primary variables)
  - 49'818 Booleans in [0,1]
#kAtMostOne: 13'386 (#literals: 1'201'014)
#kExactlyOne: 360 (#literals: 49'680)
#kLinMax: 138 (#expressions: 49'680)
#kLinear1: 32'876

Starting presolve at 0.05s
Large number of fixed variables 32'876 / 49'818, doing a first remapping phase to go down to 16'942 variables.
TODO: 32876 fixed variables still required in the model!


  1.92e-02s  0.00e+00d  [DetectDominanceRelations] 
  1.01e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  8.06e-04s  0.00e+00d  [ExtractEncodingFromLinear] #potential_supersets=13'746 


  1.49e-02s  0.00e+00d  [DetectDuplicateColumns] 
  4.25e-03s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=2'604 
[Symmetry] Graph for symmetry has 94'844 nodes and 415'651 arcs.


[Symmetry] Symmetry computation done. time: 0.0207552 dtime: 0.0356038
[SAT presolve] num removable Booleans: 0 / 16942
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:16804 literals:33608 vars:16942 one_side_vars:16942 simple_definition:0 singleton_clauses:0
[SAT presolve] [0.000507633s] clauses:16804 literals:33608 vars:16942 one_side_vars:16942 simple_definition:0 singleton_clauses:0
[SAT presolve] [0.000720006s] clauses:16804 literals:33608 vars:16942 one_side_vars:16942 simple_definition:0 singleton_clauses:0


...

#17      3.78s best:14327 next:[0,14326]  ls_lin_restart(batch:1 lin{mvs:13'056 evals:16'942} #w_updates:4'079 #perturb:0)


#18      7.75s best:14325 next:[0,14324]  ls_restart_perturb(batch:1 lin{mvs:5'314 evals:16'942} #w_updates:1'413 #perturb:0)


#19     24.25s best:14324 next:[0,14323]  rnd_var_lns (d=3.65e-01 s=970 t=0.10 p=0.50 stall=113 h=base)


#Done   27.85s max_lp
#Done   27.85s quick_restart_no_lp



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [  26.50s,   26.50s]   26.50s   0.00ns   26.50s         1 [  56.57s,   56.57s]   56.57s   0.00ns   56.57s
           'default_lp':         1 [  26.50s,   26.50s]   26.50s   0.00ns   26.50s         1 [  57.42s,   57.42s]   57.42s   0.00ns   57.42s
     'feasibility_pump':         1 [257.33ms, 257.33ms] 257.33ms   0.00ns 257.33ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         1 [ 73.54ms,  73.54ms]  73.54ms   0.00ns  73.54ms         1 [  7.16ms,   7.16ms]   7.16ms   0.00ns   7.16ms
            'fs_random':         1 [ 88.85ms,  88.85ms]  88.85ms   0.00ns  88.85ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra